# Data Analysis with Visual Analytics
**Combining some basic analytics with visualization**

NOTE: goal is to show benifit of easy RAPIDS framework compatability, and speed of which can do iterations on large(ish) dataset

## Overview and Requirements
Super short version of intro notebook and restate requirments

## Import

In [ ]:
import cudf
import cugraph
import cupy
import cuspatial

import numpy as np
import pandas as pd

import datashader as ds
import datashader.transfer_functions as tf

import hvplot.cudf
import hvplot.pandas

## Load Cleaned Data / Check

In [ ]:
from pathlib import Path

DATA_DIR = Path("./data")

In [ ]:
df = cudf.read_csv(DATA_DIR / "data.csv", parse_dates=('starttime', 'stoptime'))

In [ ]:
stations = pd.read_csv(DATA_DIR / "stations.csv")

## Run some analysis -> ? = profit?
Mix of cuML / cuspatial analysis to then visualize

## Intro to cuGraph / cuSpatial / cupy

We will analyze the data using some RAPIDS tools:

* [cuGraph](https://docs.rapids.ai/api/cugraph/stable/) is a collection of GPU accelerated graph algorithms that process data found in GPU DataFrames. We can use to compute pagerank or degree measure on the trip graph. 

* [cuSpatial](https://docs.rapids.ai/api/cuspatial/nightly/) is a collection of GPU accelerated algorithms for computing geo-spatial measures. We can use to to compute trips with given bounding regions.

## Spatial

Let's take a look at some spatial measures and see if there are any interesting features.

We might start with the first station, and see what the min/max trip length from it are.

In [ ]:
r0 = df.iloc[0]
origin_lon, origin_lat = r0["longitude_start"], r0["latitude_start"]

The cuSpatial function `lonlat_to_cartesian` will let us quicly compute the x/y distances for every ending trip location:

In [ ]:
dist = cuspatial.lonlat_to_cartesian(origin_lon[0], origin_lat[0], df["longitude_end"], df["latitude_end"])

CuPy functions can compute derived values on these GPU dataframes:

In [ ]:
cupy.sqrt(cupy.max(dist.x**2 + dist.y**2))

What if we want to compute this max/min trip interval for every station as a starting point?

In [ ]:
from_intervals = []

for idx, row in stations.iterrows():
    station_id, origin_lon, origin_lat = int(row["station_id"]), row["lon"], row["lat"]
    dist = cuspatial.lonlat_to_cartesian(origin_lon, origin_lat, df["longitude_end"], df["latitude_end"])
    from_intervals.append((station_id, float(cupy.sqrt(cupy.max(dist.x**2 + dist.y**2)))))

from_intervals = pd.DataFrame(from_intervals, columns=["station_id", "dist"])

In [ ]:
from_longest = from_intervals.nlargest(25, "dist")

In [ ]:
from_longest_loc = stations[stations.station_id.isin(from_longest.station_id)]

In [ ]:
from_longest_loc.hvplot.points(x='lon', y='lat', size=300, geo=True, tiles="OSM").opts(width=800, height=800)

## Pagerank

We will use the `cugraph.pagerank` function to see if there are patterns for the "most popular" stations. 

In [ ]:
df["weekday"] = df['starttime'].dt.weekday

### Single hour 

Let's see what it looks like to compute page range for a single hour of the day, e.g. 5PM. First subset the data to only look at data for trips starting at that hour:

In [ ]:
d17 = df[df["hour"]==17]

Next group by (from_station_id, to_station_id) and then take the group size to get all the unique indivual routes between stations that hour, and also the number of trips that took each of those routes:/

In [ ]:
g17 = df.groupby(by=["from_station_id", "to_station_id"])
routes17 = g17.size().reset_index()
routes17.head()

Now we can create a `cugraph.Graph` 

In [ ]:
G = cugraph.Graph()

In [ ]:
G.from_cudf_edgelist(d17, source='from_station_id', destination='to_station_id')

In [ ]:
d17_page = cugraph.pagerank(G)
d17_page.head()

In [ ]:
d17_top = d17_page.nlargest(10, "pagerank").to_pandas()
d17_top.head()

In [ ]:
d17_page_locs = stations[stations.station_id.isin(d17_top.vertex)]
d17_page_locs.hvplot.points(x='lon', y='lat', size=300, geo=True, tiles="OSM").opts(width=800, height=800)

Let's repeat the same  process and see the highest ranked stations at noon

In [ ]:
d8 = df[df["hour"]==12]
g8 = df.groupby(by=["from_station_id", "to_station_id"])
routes17 = g8.size().reset_index()

G = cugraph.Graph()
G.from_cudf_edgelist(d8, source='from_station_id', destination='to_station_id')
d8_page = cugraph.pagerank(G)
d8_page.head()

d8_top = d8_page.nlargest(10, "pagerank").to_pandas()

d8_page_locs = stations[stations.station_id.isin(d8_top.vertex)]
d8_page_locs.hvplot.points(x='lon', y='lat', size=300, geo=True, tiles="OSM").opts(width=800, height=800)

Now let's look at how stations rank week on weekdays vs weekends. The code below computes the pagerank broken out by individual day of the week.

In [ ]:
results = {}
for w in range(7):
    dfw = df[df["weekday"]==w]
    G = cugraph.Graph()
    G.from_cudf_edgelist(dfw, source='from_station_id', destination='to_station_id')
    df_page = cugraph.pagerank(G).nlargest(20, "pagerank")
    results[w] = set(df_page.to_pandas()["vertex"])

Now let's find out what stations were highest ranked among all weekdays and weekend days

In [ ]:
weekday = set.intersection(*[results[i] for i in range(5)])
weekend = set.intersection(results[5], results[6])

In [ ]:
weekend

In [ ]:
weekday

Finally we can plot these quickly using `hvplot`. Let's add a column to denote weekday/weekend so that we can group by that

In [ ]:
r1 = stations[stations.station_id.isin(weekend)]
r1 = r1.assign(type="Weekend")

r2 = stations[stations.station_id.isin(weekday)]
r2 = r2.assign(type="Weekday")

result = pd.concat([r1, r2])

In [ ]:
result.hvplot.points(x='lon', y='lat', by='type', alpha=0.5, size=485, geo=True, tiles="OSM").opts(width=800, height=800)

## Summary of interesting analytics results 
Does not have to be significant but noteable

In order to group by day, we first take the "floor" of each timestamp divided by one day

In [ ]:
one_day = np.datetime64(1, 'D').astype('datetime64[ns]').astype('int64') 

df['from_day'] = df['starttime'].astype('int64') // one_day
df['to_day'] = df['stoptime'].astype('int64') // one_day

Now we can group by the station id and hour for both the departing and arriving cases. We name the columns from the size DataFrame `out` and `in` respectively:

In [ ]:
df_out = df.groupby(by=["from_station_id", "from_day"]).size().to_frame('out').reset_index()
df_in = df.groupby(by=["to_station_id", "to_day"]).size().to_frame('in').reset_index()

Let's re name the columns to be the same in both DataFrames

In [ ]:
df_out.rename(columns={"from_station_id": "station_id", "from_day": "day"}, inplace=True)
df_in.rename(columns={"to_station_id": "station_id", "to_day": "day"}, inplace=True)

And re-set the index to be the (station id, hour) pair

In [ ]:
df_out = df_out.set_index(["station_id", "day"])
df_in = df_in.set_index(["station_id", "day"])

Now we can join these two DataFrames to compute an `flow = out - in` column

In [ ]:
full_df = df_in.join(df_out, how="outer").fillna(0).reset_index()
full_df["flow"] = full_df["out"] - full_df["in"]

In [ ]:
full_df["time"] = (full_df["day"] * one_day).astype('datetime64[ns]')
full_df = full_df[["station_id", "time", "flow"]]

In [ ]:
full_df.head()

In [ ]:
flows = []
for i in stations.station_id:
    subdf = full_df[full_df.station_id==i].drop("station_id").set_index("time")
    flows.append((i, subdf.flow.max(), subdf.flow.min()))
flows = pd.DataFrame(flows, columns=["station_id", "max_out", "max_in"])

In [ ]:
flows

In [ ]:
flows.iloc[flows.max_out.argmax()]

In [ ]:
flows.iloc[flows.max_in.argmin()]

In [ ]:
flows[flows.station_id.isin(weekday)]

In [ ]:
flows[flows.station_id.isin(weekday)]

In [ ]:
series = []

for i in stations.station_id:
    s = full_df[full_df.station_id==i][["time", "flow"]]
    s.rename(columns={"flow": f"s{i}"}, inplace=True)
    s = s.set_index("time")
    series.append(s)
    
df_wide = cudf.concat(series, axis=1).fillna(0)

In [ ]:
df_wide

In [ ]:
df_wide.hvplot(y=["s81", "s287"], alpha=0.2)

In [ ]:
df_wide.hvplot(y=[f"s{n}" for n in weekday], alpha=0.2)

In [ ]:
df_wide.hvplot(y=[f"s{n}" for n in weekend], alpha=0.2)

In [ ]:
len(flows[flows.max_out > 20])

In [ ]:
len(flows[flows.max_in < -20])

In [ ]:
df_cumulative = df_wide.cumsum()

In [ ]:
df_cumulative

In [ ]:
def series_shade(df):
    cols = list(df.columns)
    
    itime = cudf.to_datetime(df.index).astype('int64')
    x_range = (itime[0], itime[-1])
    
    y_range = (df.min().min(), df.max().max())
    
    temp = cudf.DataFrame(df)
    temp["itime"] = itime
    
    cvs = ds.Canvas(plot_height=400, plot_width=1000)
    agg = cvs.line(temp, x="itime", y=cols, agg=ds.count(), axis=1)
    
    return tf.shade(agg, how='eq_hist')

In [ ]:
series_shade(df_wide)

In [ ]:
series_shade(df_cumulative)

In [ ]:
df[df["to_station_id"]==268]